# OT for the Kyle model with insider activism

In [ ]:
from __future__ import division

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ot_for_kyle_activism import sinkhorn as sinkhorn

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import ot_for_kyle_activism.core.sinkhorn as kyle

## Definition of the parameters

In [ ]:
# Hyperparameters
hyperparameters = {
    'sigma'     : 1.0,
    'T'         : 1.0,
    'm_beta'    : 0.0,
    'sigma_beta': 1.0,
    'delta'     : 1.0,
    'x_star'    : 0.5
}

# Sampling parameters in grid
N = 100
M = 80
m = 4

# Sinkorn regularization parameter
epsilon = 0.05

# Solve Sinkhorn
potential_a, K1, potential_b = kyle.solve_sinkhorn(M,N,m,epsilon=epsilon, params=hyperparameters)
# Form transport plan
plan = potential_a[:, None, None]*K1*potential_b[None, :, :]

In [ ]:
kyle.plot_level_set(plan, M, N, m)
kyle.plot_level_set2(plan, M, N, m)

# II. Plot in 3D in plotly by layering Y

In [ ]:
# Solve Sinkhorn
potential_a, K1, potential_b = kyle.solve_sinkhorn(M,N,m,epsilon=epsilon, params=hyperparameters)
# Form transport plan
plan = potential_a[:, None, None]*K1*potential_b[None, :, :]

In [ ]:
import plotly.graph_objects as go

# Define grids for variables y,z,xi
grid_y  = np.linspace(-m,m,N)
grid_z  = np.linspace(-m,m,N)
grid_xi = np.linspace(0,1,M)

#
height = np.zeros( (N,N) )
for i in range(N):
    # Slice for fixed y
    # Note: Slice = Conditionnal distribution (Z,Xi | Y)
    plan_slice = plan[i,:,:]
    plan_slice = plan_slice/np.sum(plan_slice)
    # Compute mean of Xi conditionally to Z
    z = grid_z
    xi = np.dot( plan_slice, grid_xi )/np.sum(plan_slice, axis=1)
    height[i,:] = xi

fig = go.Figure( data=[
    go.Surface(x=grid_y, y=grid_z, z=height)
] )
fig.update_layout(title_text="Y = I(Z,Xi)",
                  scene = dict(
                     xaxis_title="Y",
                     yaxis_title="Z",
                     zaxis_title="Xi",
                     xaxis = dict(nticks=10, range=[-m,m],),
                     yaxis = dict(nticks=10, range=[-m,m],),
                     zaxis = dict(nticks=4, range=[0,1],),
                  )
                 )
fig.show()

# III. Plot in 3D using conditioning for computing I

In [ ]:
# Hyperparameters
hyperparameters = {
    'sigma'     : 1.0,
    'T'         : 1.0,
    'm_beta'    : 0.0,
    'sigma_beta': 1.0,
    'delta'     : 1.0,
    'x_star'    : 0.5
}


# Sampling parameters in grid
N = 100
M = 80
m = 4

# Sinkorn regularization parameter
epsilon = 0.05

# Loop over hyperparameters
Is = []
for param in [-1.0, 0.0, 1.0]:
    params=dict(hyperparameters)
    params['m_beta'] = param
    # Solve Sinkhorn
    potential_a, K1, potential_b = kyle.solve_sinkhorn(M,N,m,epsilon=epsilon, params=params)
    # Form transport plan
    plan = potential_a[:, None, None]*K1*potential_b[None, :, :]
    # Compute I
    grid_z, grid_y, height = kyle.compute_I( plan, M, N, m)
    Is.append( height )


import plotly.graph_objects as go

data = []
for I in Is:
    data.append( go.Surface(x=grid_z, y=grid_xi, z=I) )
fig = go.Figure( data=data )
fig.update_layout(title_text="Y = I(Z,Xi)",
                  scene = {
                     'xaxis_title': 'Z',
                     'yaxis_title': 'Xi',
                     'zaxis_title': 'Y',
                     'xaxis': { 'nticks':10, 'range': [-m,m] },
                     'yaxis': { 'nticks':10, 'range': [0,1]  },
                     'zaxis': { 'nticks':10, 'range': [-m,m] },
                  })
fig.write_image("multiple_m.png")
fig.show()


# IV. Probability of taking a role in governance

In [ ]:
def compute_governance_probabilities( M, N, m, epsilon, hyperparameters, sigma_array):
    probabilities = []
    for param in sigma_array:
        params=dict(hyperparameters)
        params['sigma'] = param
        # Solve Sinkhorn
        potential_a, K1, potential_b = kyle.solve_sinkhorn(M,N,m,epsilon=epsilon, params=params)
        # Form transport plan
        plan = potential_a[:, None, None]*K1*potential_b[None, :, :]
        # Define grids for variables y,z,xi
        grid_y  = np.linspace(-m,m,N)
        grid_z  = np.linspace(-m,m,N)
        grid_xi = np.linspace(0,1,M)
        # Normalize for safety
        plan = plan/np.sum(plan)
        # Law of (Y,Z)
        law_YZ = np.sum( plan, axis=2 )
        #
        y, z = np.meshgrid( grid_y, grid_z )
        indices = np.where( y-z>hyperparameters['x_star'])
        p = np.sum( law_YZ[indices])
        probabilities.append( p )
    # End for
    return probabilities

# Hyperparameters
hyperparameters = {
    'sigma'     : 1.0,
    'T'         : 1.0,
    'm_beta'    : -1.0,
    'sigma_beta': 1.0,
    'delta'     : 1.0,
    'x_star'    : 0.5
}


# Sampling parameters in grid
N = 100
M = 80
m = 4

# Sinkorn regularization parameter
epsilon = 0.05

sigma_array=np.linspace(0.1, 2.0, 20)

#
hyperparameters['m_beta'] = -1.0
probabilities1 = compute_governance_probabilities( M, N, m, epsilon, hyperparameters, sigma_array)
plt.show()
hyperparameters['m_beta'] = 1.0
probabilities2 = compute_governance_probabilities( M, N, m, epsilon, hyperparameters, sigma_array)
plt.show()



In [ ]:
# Plot
plt.figure(figsize=(10, 8), dpi=80)
matplotlib.rcParams.update({'font.size': 15})
plt.plot( sigma_array, probabilities1, label='$m_\\beta$=1.0' )
plt.plot( sigma_array, probabilities2, label='$m_\\beta$=1.0' )
plt.legend()
plt.title( 'Probability $\mathbb{P}( Y_T-(Z_T-\\beta) \geq x^*)$ as a function of $\sigma$.')
plt.xlabel('$\\sigma$')
plt.ylabel('$\\mathbb{P}( Y_T-(Z_T-\\beta) \geq x^*)$')
plt.ylim( [0,1] )
plt.savefig('proba.png')
plt.show()